In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataindoml/massive_train.solution
/kaggle/input/dataindoml/massive_train.data


In [2]:
!pip install transformers torch

In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0
    Uninstalling accelerate-0.22.0:
      Successfully uninstalled accelerate-0.22.0


In [4]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# Load the data from the JSON files
with open('/kaggle/input/dataindoml/massive_train.data', 'r') as data_file:
    data = [json.loads(line) for line in data_file]

with open('/kaggle/input/dataindoml/massive_train.solution', 'r') as solution_file:
    solutions = [json.loads(line) for line in solution_file]

In [6]:
print(data[0])

{'indoml_id': 'af-ZA|1', 'id': '1', 'locale': 'af-ZA', 'partition': 'train', 'scenario': 'alarm', 'utt': 'maak my wakker nege-uur v. m. op vrydag', 'annot_utt': 'maak my wakker [time : nege-uur v. m.] op [date : vrydag]', 'worker_id': '20', 'slot_method': [{'slot': 'time', 'method': 'translation'}, {'slot': 'date', 'method': 'translation'}], 'judgments': [{'worker_id': '40', 'intent_score': 1, 'slots_score': 1, 'grammar_score': 4, 'spelling_score': 2, 'language_identification': 'target'}, {'worker_id': '49', 'intent_score': 1, 'slots_score': 1, 'grammar_score': 4, 'spelling_score': 2, 'language_identification': 'target'}, {'worker_id': '20', 'intent_score': 1, 'slots_score': 1, 'grammar_score': 4, 'spelling_score': 2, 'language_identification': 'target'}]}


In [7]:
print(solutions[0])

{'indoml_id': 'af-ZA|1', 'intent': 'alarm_set'}


In [8]:
# Create a dictionary to map indoml_id to intents
intent_map = {item['indoml_id']: item['intent'] for item in solutions}

# Split data into train and test sets (2:1) stratified by intent
indoml_ids = [item['indoml_id'] for item in data]
intents = [intent_map[indoml_id] for indoml_id in indoml_ids]

utt = [item['utt'] for item in data]

num_classes = len(set(intents))
print("Number of classes")
print(num_classes)

train_data, test_data, train_labels, test_labels = train_test_split(
    utt, intents, test_size=0.33, random_state=42, stratify=intents
)



Number of classes
60


In [9]:
pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 53.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.0 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which 

In [10]:
# Define the BERT model and tokenizer
# keep `ignore_mismatched_sizes=True` so that the classification layer is randomly initialized
model_name = "cartesinus/bert-base-uncased-amazon-massive-intent"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)

In [11]:
# Tokenize the input data
train_encodings = tokenizer(
    train_data,
    truncation=True,
    padding=True,
    max_length=32,
    return_tensors='pt'    #return type is pytorch tensor
)

test_encodings = tokenizer(
    test_data,
    truncation=True,
    padding=True,
    max_length=32,
    return_tensors='pt'
)

In [12]:
# Convert labels to numeric values
label_map = {intent: i for i, intent in enumerate(set(train_labels))}
train_labels = [label_map[label] for label in train_labels]
test_labels = [label_map[label] for label in test_labels]

In [13]:
# Create PyTorch datasets
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(train_encodings, train_labels)
test_dataset = IntentDataset(test_encodings, test_labels)

In [14]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./intent_classification',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch", # use 'epoch' for evaluating every epoch
    logging_steps=10000,
    eval_steps=10000,
    save_total_limit=5,
    learning_rate=1e-4,
    num_train_epochs=3,
    logging_dir='./logs',
)

In [ ]:
# Create a trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,2.173400,1.520643


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Predict on the test set
raw_predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(raw_predictions.predictions, axis=1)

# Convert labels back to original intents
predicted_intents = [list(label_map.keys())[list(label_map.values()).index(label)] for label in predicted_labels]

# Print one sample prediction
sample_idx = random.randint(0, len(test_data) - 1)
sample_text = test_data[sample_idx]
sample_intent = predicted_intents[sample_idx]

print(f"Sample Text: {sample_text}")
print(f"Predicted Intent: {sample_intent}")

In [ ]:
# Calculate accuracy, precision, recall, and F1-score
report = classification_report(test_labels, predicted_labels, target_names=list(label_map.keys()), output_dict=True)

accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")